In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
from utils import quantise_model,get_model_memory_size, compute_quantisation_mse, Perplexity
from datasets import load_dataset
import torch
from tqdm import tqdm


def compute_perplexity(chunk_size = None):

    tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
    model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

    text = [t for t in load_dataset("wikitext", "wikitext-2-raw-v1", split="test")['text'] if len(t) > 30]

    if chunk_size:
        get_model_memory_size(model)
        model, parameter_mapping = quantise_model(model, chunk_size=chunk_size)
        get_model_memory_size(model, parameter_mapping)


    perplexity = Perplexity()
    p = perplexity._compute(text, model, tokenizer, batch_size=16)

    del model
    torch.cuda.empty_cache()
    return p['mean_perplexity']

d = []
for chunk in [32, 64, 128, None]:

    d.append(compute_perplexity(chunk_size=chunk))

print(d)




model size: 497759232 / bit | 497.76 / MB


Quantising model layers: 100%|██████████| 148/148 [00:00<00:00, 314.82it/s]


model size: 165199104 / bit | 165.20 / MB


  0%|          | 0/145 [00:00<?, ?it/s]

model size: 497759232 / bit | 497.76 / MB


...

Quantising model layers: 100%|██████████| 148/148 [00:00<00:00, 338.96it/s]


model size: 134629632 / bit | 134.63 / MB


  0%|          | 0/145 [00:00<?, ?it/s]

  0%|          | 0/145 [00:00<?, ?it/s]

[174.04702233875474, 177.15088416117325, 187.7257970190689, 159.90127864998038]


In [ ]:
import torch
from utils import quantise_tensor, dequantise_tensor, compute_quantisation_mse,get_tensor_memory_size

t = torch.rand((10, 100))
chunk_size = 256
t_q, scales, locations = quantise_tensor(t, chunk_size)
t_approx = dequantise_tensor(t_q, scales, locations, chunk_size)

mse = compute_quantisation_mse(t, t_approx)

print(f'Required memory: {get_tensor_memory_size(t)} vs {get_tensor_memory_size(t_q) + get_tensor_memory_size(scales) + get_tensor_memory_size(locations)}')